## Finetuned model evaluation: 

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
# !pip install datasets>=2.6.1
# !pip install git+https://github.com/huggingface/transformers
# !pip install librosa
# !pip install evaluate>=0.30
# !pip install jiwer
# !pip install gradio

In [2]:
from huggingface_hub import notebook_login

notebook_login()

## Load Dataset

In [4]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import Dataset, load_dataset
import torchaudio
import torch
from jiwer import wer  # Install jiwer using: pip install jiwer

# Load the Whisper processor and model
processor = WhisperProcessor.from_pretrained("ZHProject23/whisper-small-vn")
model = WhisperForConditionalGeneration.from_pretrained("ZHProject23/whisper-small-vn")
model.config.forced_decoder_ids = None

# Replace "/path/to/folder" with the actual path to your data folder
data_folder_path = "./data/all/"

# Load the audiofolder dataset
dataset = load_dataset("audiofolder", data_dir=data_folder_path)

# Shuffle the dataset
shuffled_dataset = dataset.shuffle(seed=123)

# Manually split the dataset into train and test
split_percentage = 0.8
split_index = int(len(shuffled_dataset["train"]) * split_percentage)

train_dataset = Dataset.from_dict(shuffled_dataset["train"][:split_index])
test_dataset = Dataset.from_dict(shuffled_dataset["train"][split_index:])

# Display available splits in the custom DatasetDict
print(train_dataset, test_dataset)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.84k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/5952 [00:00<?, ?it/s]

Dataset({
    features: ['audio', 'transcription'],
    num_rows: 2364
}) Dataset({
    features: ['audio', 'transcription'],
    num_rows: 592
})


## Calculating WER on test set: 

In [5]:
import re
from vietnam_number import n2w

def preprocess_text(input_text):
    # Task 1: Lowercase all words
    lowercased_text = input_text.lower()

    # Task 2: Remove all non-letter characters (except numbers, %, and dates)
    # Preserve characters with Vietnamese tone marks
    non_letter_pattern = r'[^a-zA-Z0-9%\/àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵ]+'
    letters_and_numbers_text = re.sub(non_letter_pattern, ' ', lowercased_text)

    # Task 3: Remove all line breaks
    cleaned_text = re.sub(r'\n', ' ', letters_and_numbers_text)

    # Task 4: Replace numbers with their full form using vietnam_number
    def replace_numbers(match):
        number = match.group(0)
        return n2w(number)

    # Replace numbers with their full form
    expanded_text = re.sub(r'\b\d+\b', replace_numbers, cleaned_text)

    # Task 5: Convert dates
    def replace_date(match):
        date = match.group(0)
        day, month, year = date.split('/')
        
        day_text = n2w(day)  # Convert day to written form using vietnam_number
        month_text = [
            'tháng một', 'tháng hai', 'tháng ba', 'tháng tư', 'tháng năm', 'tháng sáu',
            'tháng bảy', 'tháng tám', 'tháng chín', 'tháng mười', 'tháng mười một', 'tháng mười hai'
        ][int(month) - 1]  # Choose the month text
        year_text = 'năm ' + n2w(year)  # Convert year to written form using vietnam_number
        
        return f'ngày {day_text} {month_text} {year_text}'

    # Replace dates with the desired format
    date_pattern = r'\b\d{1,2}/\d{1,2}/\d{4}\b'
    expanded_text = re.sub(date_pattern, replace_date, expanded_text)

    # Task 6: Add a space between numbers and "%" symbols
    expanded_text = re.sub(r'%', r' phần trăm', expanded_text)

    return expanded_text


# Inference on the test dataset
reference_texts = []
hypothesis_texts = []
count = 1

for example in test_dataset:
    # Load audio file
    waveform, sample_rate = torchaudio.load(example["audio"]["path"], normalize=True)

    # Preprocess audio
    input_features = processor(waveform.numpy()[0], sampling_rate=sample_rate, return_tensors="pt").input_features

    # Generate token ids
    with torch.no_grad():
        predicted_ids = model.generate(input_features)

    # Decode token ids to text
    hypothesis = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    hypothesis = preprocess_text(hypothesis)

    # Load reference transcription
    # transcription_path = example["transcription"]
    # with open(transcription_path, "r", encoding="utf-8") as file:
    #     reference = file.read().strip()
    reference = example["transcription"]

    # Append results to lists
    reference_texts.append(reference)
    hypothesis_texts.append(hypothesis)
    print("test case: ", count)
    count += 1
    print("hypothesis: ", hypothesis)
    print("reference: ", reference)
    print("wer: ", wer(reference,hypothesis))

# Calculate Word Error Rate (WER)
wer_score = wer(reference_texts, hypothesis_texts)

print(f"Word Error Rate on Test Data: {wer_score:.2%}")


test case:  1
hypothesis:  giàn đã trung tín tường thuật mọi điều ông đã chứng kiến
reference:  giăng đã trung tín tường thuật mọi điều ông đã chứng kiến
wer:  0.08333333333333333
test case:  2
hypothesis:  như một số báo chí đã bình luận trong thời gian qua
reference:  như một số báo chí đã bình luận trong thời gian qua
wer:  0.0
test case:  3
hypothesis:  làm lòng tự trọng bị xây xước nghiêm trọng
reference:  làm lòng tự trọng bị xây xước nghiêm trọng
wer:  0.0
test case:  4
hypothesis:  do đó niềm hy vọng của chúng ta đầy sức sống vì chúa cứu thế đã từ cõi chúa
reference:  do đó niềm hy vọng của chúng ta đầy sức sống vì chúa cứu thế đã từ cõi chết
wer:  0.05263157894736842
test case:  5
hypothesis:  chúng ta càng chịu khổ đau vì chúa cứu thế bao nhiêu ngài càng an uổi giúp chúng ta
reference:  chúng ta càng chịu khổ đau vì chúa cứu thế bao nhiêu ngài càng an ủi giúp chúng ta
wer:  0.05263157894736842
test case:  6
hypothesis:  nhưng chị em chỉ lo đùa dẫn cười cờ và ăn cho đầy bụng c

## Calculating CER on test set:

In [13]:
from evaluate import load
cer = load("cer")

# Inference on the test dataset
cer_reference_texts = []
cer_hypothesis_texts = []
count = 1

for example in test_dataset:
    # Load audio file
    waveform, sample_rate = torchaudio.load(example["audio"]["path"], normalize=True)

    # Preprocess audio
    input_features = processor(waveform.numpy()[0], sampling_rate=sample_rate, return_tensors="pt").input_features

    # Generate token ids
    with torch.no_grad():
        predicted_ids = model.generate(input_features)

    # Decode token ids to text
    hypothesis = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    hypothesis = preprocess_text(hypothesis)

    # Load reference transcription
    # transcription_path = example["transcription"]
    # with open(transcription_path, "r", encoding="utf-8") as file:
    #     reference = file.read().strip()
    reference = example["transcription"]

    # Append results to lists
    cer_reference_texts.append(reference)
    cer_hypothesis_texts.append(hypothesis)
    print("test case: ", count)
    count += 1
    print("hypothesis: ", hypothesis)
    print("reference: ", reference)
    cer_score = cer.compute(predictions=[hypothesis], references=[reference])
    print("cer: ", cer_score)

# Calculate Word Error Rate (WER)
cer_score = cer.compute(predictions=cer_hypothesis_texts, references=cer_reference_texts)

print(f"CER on Test Data: {cer_score:.2%}")


test case:  1
hypothesis:  giàn đã trung tín tường thuật mọi điều ông đã chứng kiến
reference:  giăng đã trung tín tường thuật mọi điều ông đã chứng kiến
cer:  0.03508771929824561
test case:  2
hypothesis:  như một số báo chí đã bình luận trong thời gian qua
reference:  như một số báo chí đã bình luận trong thời gian qua
cer:  0.0
test case:  3
hypothesis:  làm lòng tự trọng bị xây xước nghiêm trọng
reference:  làm lòng tự trọng bị xây xước nghiêm trọng
cer:  0.0
test case:  4
hypothesis:  do đó niềm hy vọng của chúng ta đầy sức sống vì chúa cứu thế đã từ cõi chúa
reference:  do đó niềm hy vọng của chúng ta đầy sức sống vì chúa cứu thế đã từ cõi chết
cer:  0.02666666666666667
test case:  5
hypothesis:  chúng ta càng chịu khổ đau vì chúa cứu thế bao nhiêu ngài càng an uổi giúp chúng ta
reference:  chúng ta càng chịu khổ đau vì chúa cứu thế bao nhiêu ngài càng an ủi giúp chúng ta
cer:  0.024390243902439025
test case:  6
hypothesis:  nhưng chị em chỉ lo đùa dẫn cười cờ và ăn cho đầy bụng 